In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns
import dash
from dash.dependencies import Input, Output, State
from dash import dcc, html, dash_table

import pandas as pd
import seaborn as sns



import dash_bootstrap_components as dbc
import matplotlib.pyplot as plt
import plotly.express as px
from jupyter_dash import JupyterDash

In [2]:
import plotly.express as px

In [3]:
df=pd.read_csv("Real estate.csv",index_col=0)
df.head()

In [5]:
feature=['X2 house age']
X=df[feature]
target='Y house price of unit area'
Y=df[target]

In [6]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.utils.validation import check_is_fitted
from sklearn.model_selection import train_test_split

In [7]:
df.columns

Index(['X1 transaction date', 'X2 house age',
       'X3 distance to the nearest MRT station',
       'X4 number of convenience stores', 'X5 latitude', 'X6 longitude',
       'Y house price of unit area'],
      dtype='object')

In [8]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout=html.Div([
    
    
    html.H1("Linear-Regression",style={"color": 'red',"textAlign": 'center'}),
    
    html.Br(),
    
    
    dbc.Row([
        dbc.Col([
            dbc.Label("Select Feature variable",style={'marginLeft' : '20px','textAlign':'center'}),
            dcc.Dropdown(id='d1',
                         options=[
                        {"label": col, "value": col} for col in df.columns],
                        value=df.columns[1],style={'margin' : '10px'})
            
        ],width=4),
        
        dbc.Col([
            dbc.Label("Select Target variable",style={'marginLeft' : '10px'}),
            dcc.Dropdown(id='d2',
                         options=[
                        {"label": col, "value": col} for col in df.columns],
                        value=df.columns[6],style={'margin' : '10px'})
        ],width=4),
        
        
    ]),
    
    html.Br(),
    html.Br(),
    html.Br(),
    
    dbc.Row([
        dbc.Col([ html.H3("Model Summary",style={"color":'blue','marginLeft': '50px'}),
            dbc.Row([html.Div(id='d3',style={"width":'66%','margin' : '30px'})]),
            
        ],width=4),
        
        
        
        dbc.Col([
            
            dcc.Graph(id='figu',style={"marginLeft": '40px'})
        ],width=8),
        
    ])
    
    
    
    
])


@app.callback(Output('figu','figure'),
             [Input('d1','value'),
             Input('d2','value')])
def f(a,b):
    feature=[a]
    X=df[feature]
    target=b
    Y=df[target]
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    
    model=make_pipeline(LinearRegression())
    
    model.fit(X_train,Y_train)
    
    Y_pred=model.predict(X_test)
    
    global mea
    
    mea=round(mean_absolute_error(Y_test,Y_pred),3)
    
    global coef
    
    coef=model.named_steps['linearregression'].coef_
    
    global intercept
    
    intercept=round(model.named_steps['linearregression'].intercept_,2)
    
    x_range = np.linspace(X.min(), X.max(), 100)
    y_range = model.predict(x_range.reshape(-1, 1))
    
    fig = px.scatter(df, x=a, y=b, opacity=0.65,trendline='ols')
    #fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Regression Fit'))
    fig.update_layout(
    
        width =900,
        height=500
    )
    
    return fig

@app.callback(Output('d3','children'),
             [
                 Input('d1','value'),
                 Input('d2','value')]
             )
def summary(a,b):
    
    if len(a)==0 and len(b)==0:
        
        raise PreventUpdate
    else:
        
    
        return (

            f"The value of coefficient is : {coef}.                       Value of intercept is : {intercept}.                                      Mean_Absolute_Error is : {mea} " 
        )







In [10]:
if __name__=='__main__':
    app.run_server(port='8092')